In [1]:
# Importing required libraries and dependencies 
import pandas as pd
import datetime as dt 
import numpy as np
from prophet import Prophet 
import matplotlib.pyplot as plt
import hvplot.pandas as hvplot


In [2]:
# Importing the data from the csv file for private sector
private_sector_df = pd.read_csv("../Resources/privatesector_Data.csv", delimiter=';').dropna()

# Displaying the data head,tail, data types and summary statistics
display(private_sector_df.head())
display(private_sector_df.tail())
display(private_sector_df.dtypes)
display(private_sector_df.describe())


,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
0,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Brazil,BRA,1970.0,1.189510e+01
2,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,India,IND,1970.0,5.556494e+00
5,Agricultural raw materials imports (% of merch...,TM.VAL.AGRI.ZS.UN,Brazil,BRA,1970.0,1.851044e+00
7,Agricultural raw materials imports (% of merch...,TM.VAL.AGRI.ZS.UN,India,IND,1970.0,9.207277e+00
64,Commercial service exports (current US$),TX.VAL.SERV.CD.WT,South Africa,ZAF,1970.0,5.739998e+08


,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
42236,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,China,CHN,2019.0,50.738701
42237,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,India,IND,2019.0,17.704861
42238,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,Russian Federation,RUS,2019.0,37.015007
42239,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,South Africa,ZAF,2019.0,20.529111
42243,Value lost due to electrical outages (% of sal...,IC.FRM.OUTG.ZS,Russian Federation,RUS,2019.0,0.300000


SeriesName      object
SeriesCode      object
CountryName     object
CountryCode     object
Year           float64
Value          float64
dtype: object

,Year,Value
count,17420.000000,1.742000e+04
mean,2001.752411,1.159949e+10
std,12.594159,1.057388e+11
min,1970.000000,0.000000e+00
25%,1994.000000,4.474096e+00
50%,2004.000000,1.660000e+01
75%,2012.000000,8.529821e+01
max,2019.000000,2.501330e+12


In [3]:
#Filter the data for the last 10 years
private_sector_df = private_sector_df.loc[private_sector_df['Year'] >= 2009]

#Displaying first 5 and last 5 rows of the data
display(private_sector_df.head())
display(private_sector_df.tail())
display(private_sector_df.describe())

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
32955,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Brazil,BRA,2009.0,3.770854
32956,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,China,CHN,2009.0,0.448398
32957,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,India,IND,2009.0,1.164823
32958,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Russian Federation,RUS,2009.0,2.301575
32959,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,South Africa,ZAF,2009.0,1.922157


,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
42236,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,China,CHN,2019.0,50.738701
42237,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,India,IND,2019.0,17.704861
42238,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,Russian Federation,RUS,2019.0,37.015007
42239,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,South Africa,ZAF,2019.0,20.529111
42243,Value lost due to electrical outages (% of sal...,IC.FRM.OUTG.ZS,Russian Federation,RUS,2019.0,0.300000


,Year,Value
count,6463.000000,6.463000e+03
mean,2014.069782,2.184586e+10
std,3.007304,1.624208e+11
min,2009.000000,0.000000e+00
25%,2012.000000,4.939012e+00
50%,2014.000000,1.731051e+01
75%,2017.000000,1.000000e+02
max,2019.000000,2.501330e+12


In [4]:
#Fetch Unique Series Names from the private_sector_df to pick the series we want to plot and forecast
private_sector_df['SeriesName'].unique()


array(['Agricultural raw materials exports (% of merchandise exports)',
       'Agricultural raw materials imports (% of merchandise imports)',
       'Average number of visits or required meetings with tax officials (for affected firms)',
       'Average time to clear exports through customs (days)',
       'Binding coverage, all products (%)',
       'Binding coverage, manufactured products (%)',
       'Binding coverage, primary products (%)',
       'Bound rate, simple mean, all products (%)',
       'Bound rate, simple mean, manufactured products (%)',
       'Bound rate, simple mean, primary products (%)',
       'Bribery incidence (% of firms experiencing at least one bribe payment request)',
       'Business extent of disclosure index (0=less disclosure to 10=more disclosure)',
       'Commercial service exports (current US$)',
       'Commercial service imports (current US$)',
       'Computer, communications and other services (% of commercial service exports)',
       'Compu

## The series Names that are picked for further analysis and plotting
* Agricultural raw materials exports (% of merchandise exports)
* Binding coverage, all products (%)
* Commercial service imports (current US$)
* Import value index (2000 = 100)
* International tourism, expenditures (current US$)
* Investment in energy with private participation (current US$)
* Profit tax (% of commercial profits)
* Tax payments (number)



In [5]:
#From the Private Sector Data, we are interested in the series "Agricultural raw materials exports (% of merchandise exports)"
#Filter the data for the series "Agricultural raw materials exports (% of merchandise exports)"

Agricultural_raw_materials_exports = private_sector_df.loc[private_sector_df['SeriesName'] == 'Agricultural raw materials exports (% of merchandise exports)']
Agricultural_raw_materials_exports.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
32955,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Brazil,BRA,2009.0,3.770854
32956,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,China,CHN,2009.0,0.448398
32957,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,India,IND,2009.0,1.164823
32958,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Russian Federation,RUS,2009.0,2.301575
32959,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,South Africa,ZAF,2009.0,1.922157


In [6]:
# Grouping the Agricultural_raw_materials_exports data by Year and country to get the mean of the values for each year
Agricultural_raw_materials_exports_grouped = Agricultural_raw_materials_exports.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Agricultural_raw_materials_exports_grouped.head(10)

,Year,CountryCode,Value
0,2009.0,BRA,3.770854
1,2009.0,CHN,0.448398
2,2009.0,IND,1.164823
3,2009.0,RUS,2.301575
4,2009.0,ZAF,1.922157
5,2010.0,BRA,3.851848
6,2010.0,CHN,0.456645
7,2010.0,IND,2.006039
8,2010.0,RUS,2.071819
9,2010.0,ZAF,1.774643


#### Agricultural raw materials exports (% of merchandise exports)

Agricultural raw materials exports refer to the export of unprocessed or minimally processed materials derived from agriculture. These materials are essential for various industries and include items such as raw cotton, wool, hides, skins, and other crude materials (excluding fuels).



In [7]:
# Creating a line plot to visualize how the minimally processed materials are exported across multiple countries throughout the years
Agricultural_raw_materials_exports_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="Agricultural raw materials exports (% of merchandise exports)", ylabel="Merchanise Export (%)")

:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)

##### Observation - 
            Minimally Processed Material exports is almost same for every country except for Brazil it increased over the last 5 years.

In [8]:
#From the Private Sector Data, we are interested in the series "Binding coverage, all products (%)"
#Filter the data for the series "Binding coverage, all products (%)"

Binding_Coverage_All_Products = private_sector_df.loc[private_sector_df['SeriesName'] == 'Binding coverage, all products (%)']
Binding_Coverage_All_Products.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
32975,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,Brazil,BRA,2009.0,100.00
32976,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,China,CHN,2009.0,100.00
32977,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,India,IND,2009.0,74.46
32978,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,Russian Federation,RUS,2009.0,100.00
32979,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,South Africa,ZAF,2009.0,96.02


In [9]:
# Grouping the data by Year and country to get the mean of the values for each year
Binding_Coverage_All_Products_grouped = Binding_Coverage_All_Products.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Binding_Coverage_All_Products.head(10)

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
32975,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,Brazil,BRA,2009.0,100.00
32976,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,China,CHN,2009.0,100.00
32977,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,India,IND,2009.0,74.46
32978,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,Russian Federation,RUS,2009.0,100.00
32979,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,South Africa,ZAF,2009.0,96.02
33820,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,Brazil,BRA,2010.0,100.00
33821,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,China,CHN,2010.0,100.00
33822,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,India,IND,2010.0,74.46
33823,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,Russian Federation,RUS,2010.0,100.00
33824,"Binding coverage, all products (%)",TM.TAX.MRCH.BC.ZS,South Africa,ZAF,2010.0,96.02


#### Binding coverage, all products (%)
Binding coverage refers to the percentage of tariff lines or products for which a country has committed to a maximum tariff rate, 
known as a bound rate, under the World Trade Organization (WTO) agreements. 
This commitment means that the country cannot increase tariffs above the bound rate without negotiating 
with its trading partners and potentially offering compensation

In [10]:
# Creating a line plot to visualize the Binding coverage, all products (%) data across multiple countries throughout the years
Binding_Coverage_All_Products_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="Binding coverage, all products (%)", ylabel="Binding Coverage (%)")

:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)

##### Observation - 
            Binding Coverage is constant across all the years for most of the countries.No Data available for Brazil and China.

In [11]:
#From the Private Sector Data, we are interested in the series "Commercial service imports (current US$)"
#Filter the data for the series "Commercial service imports (current US$)"

Commercial_service_imports = private_sector_df.loc[private_sector_df['SeriesName'] == 'Commercial service imports (current US$)']
Commercial_service_imports.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
33020,Commercial service imports (current US$),TM.VAL.SERV.CD.WT,Brazil,BRA,2009.0,4.407476e+10
33021,Commercial service imports (current US$),TM.VAL.SERV.CD.WT,China,CHN,2009.0,1.451394e+11
33022,Commercial service imports (current US$),TM.VAL.SERV.CD.WT,India,IND,2009.0,5.230854e+10
33023,Commercial service imports (current US$),TM.VAL.SERV.CD.WT,Russian Federation,RUS,2009.0,6.120869e+10
33024,Commercial service imports (current US$),TM.VAL.SERV.CD.WT,South Africa,ZAF,2009.0,1.498016e+10


In [12]:
# Grouping the data by Year and country to get the mean of the values for each year
Commercial_service_imports_grouped = Commercial_service_imports.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Commercial_service_imports_grouped.head(10)

,Year,CountryCode,Value
0,2009.0,BRA,4.407476e+10
1,2009.0,CHN,1.451394e+11
2,2009.0,IND,5.230854e+10
3,2009.0,RUS,6.120869e+10
4,2009.0,ZAF,1.498016e+10
5,2010.0,BRA,5.781259e+10
6,2010.0,CHN,1.397866e+11
7,2010.0,IND,7.821086e+10
8,2010.0,RUS,7.322561e+10
9,2010.0,ZAF,1.915843e+10


#### Commercial service imports (current US$)
Commercial service imports refer to the total value of services imported by a country, excluding government services not included elsewhere. 
These services can include a wide range of activities such as transportation, travel, financial services, insurance, telecommunications, and business services

In [13]:

# Creating a line plot to visualize the data for the series "Commercial service imports (current US$)" across multiple countries throughout the years
Commercial_service_imports_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="Commercial service imports (current US$)", ylabel="Commercial Service")



:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)

##### Observation - 
            Commercial Service Imports increased a lot for China compared with Rest of the countries.
            for Brazil and South Africa the Commercial service Imports have not increased and are almost same across the years.

In [14]:
#From the Private Sector Data, we are interested in the series "Import value index (2000 = 100)"
#Filter the data for the series "Import value index (2000 = 100)"

Import_Value_Index = private_sector_df.loc[private_sector_df['SeriesName'] == 'Import value index (2000 = 100)']
Import_Value_Index.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
33190,Import value index (2000 = 100),TM.VAL.MRCH.XD.WD,Brazil,BRA,2009.0,227.949600
33191,Import value index (2000 = 100),TM.VAL.MRCH.XD.WD,China,CHN,2009.0,447.029206
33192,Import value index (2000 = 100),TM.VAL.MRCH.XD.WD,India,IND,2009.0,499.199773
33193,Import value index (2000 = 100),TM.VAL.MRCH.XD.WD,Russian Federation,RUS,2009.0,427.540012
33194,Import value index (2000 = 100),TM.VAL.MRCH.XD.WD,South Africa,ZAF,2009.0,249.380367


In [15]:
# Grouping the data by Year and country to get the mean of the values for each year
Import_Value_Index_grouped = Import_Value_Index.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Import_Value_Index_grouped.head(10)

,Year,CountryCode,Value
0,2009.0,BRA,227.949600
1,2009.0,CHN,447.029206
2,2009.0,IND,499.199773
3,2009.0,RUS,427.540012
4,2009.0,ZAF,249.380367
5,2010.0,BRA,326.614168
6,2010.0,CHN,620.488037
7,2010.0,IND,679.761426
8,2010.0,RUS,554.219607
9,2010.0,ZAF,326.100169


#### Import value index (2000 = 100)
This is a measure used to track changes in the value of a country’s imports over time, using the year 2000 as the base year (index value of 100). This index helps to understand how the value of imports has evolved, taking into account factors like price changes and currency fluctuations

In [16]:

# Creating a line plot to visualize the data for the series "Import value index (2000 = 100)" across multiple countries throughout the years
Import_Value_Index_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="Import value index (2000 = 100)", ylabel="Import Value Index")


:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)

##### Observation - 
            The Countries imports over time have decresed for all the countries during 2014 - 2016 and started to increase from 2016 onwards 

In [17]:
#From the Private Sector Data, we are interested in the series "International tourism, expenditures (current US$)"
#Filter the data for the series "International tourism, expenditures (current US$)"

Internation_tourism_Expenditures = private_sector_df.loc[private_sector_df['SeriesName'] == 'International tourism, expenditures (current US$)']
Internation_tourism_Expenditures.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
33220,"International tourism, expenditures (current US$)",ST.INT.XPND.CD,Brazil,BRA,2009.0,1.289700e+10
33221,"International tourism, expenditures (current US$)",ST.INT.XPND.CD,China,CHN,2009.0,4.370200e+10
33222,"International tourism, expenditures (current US$)",ST.INT.XPND.CD,India,IND,2009.0,9.310000e+09
33223,"International tourism, expenditures (current US$)",ST.INT.XPND.CD,Russian Federation,RUS,2009.0,2.378500e+10
33224,"International tourism, expenditures (current US$)",ST.INT.XPND.CD,South Africa,ZAF,2009.0,6.421000e+09


In [18]:
# Grouping the data by Year and country to get the mean of the values for each year
Internation_tourism_Expenditures_grouped = Internation_tourism_Expenditures.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Internation_tourism_Expenditures_grouped.head(10)

,Year,CountryCode,Value
0,2009.0,BRA,1.289700e+10
1,2009.0,CHN,4.370200e+10
2,2009.0,IND,9.310000e+09
3,2009.0,RUS,2.378500e+10
4,2009.0,ZAF,6.421000e+09
5,2010.0,BRA,1.888300e+10
6,2010.0,CHN,5.488000e+10
7,2010.0,IND,1.049000e+10
8,2010.0,RUS,3.016900e+10
9,2010.0,ZAF,8.139000e+09


#### International tourism, expenditures (current US$)
International tourism expenditures refer to the total amount of money spent by residents of a country on travel and tourism-related activities abroad. This includes expenses on accommodation, food, transportation, entertainment, and other services while traveling internationally.

In [19]:

# Creating a line plot to visualize the data for the series "International tourism, expenditures (current US$)" across multiple countries throughout the years
Internation_tourism_Expenditures_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="International tourism, expenditures (current US$)", ylabel="International tourism, expenditures")



:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)

##### Observation - 
            The chinese tourism expenditure have grown drastically from 2012 compared with rest of the countries

In [20]:
#From the Private Sector Data, we are interested in the series "Profit tax (% of commercial profits)"

Profit_Tax_Commercial_Percent = private_sector_df.loc[private_sector_df['SeriesName'] == 'Profit tax (% of commercial profits)']
Profit_Tax_Commercial_Percent.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
33544,Profit tax (% of commercial profits),IC.TAX.PRFT.CP.ZS,South Africa,ZAF,2009.0,24.9
34389,Profit tax (% of commercial profits),IC.TAX.PRFT.CP.ZS,South Africa,ZAF,2010.0,24.8
35234,Profit tax (% of commercial profits),IC.TAX.PRFT.CP.ZS,South Africa,ZAF,2011.0,24.0
36079,Profit tax (% of commercial profits),IC.TAX.PRFT.CP.ZS,South Africa,ZAF,2012.0,23.9
36920,Profit tax (% of commercial profits),IC.TAX.PRFT.CP.ZS,Brazil,BRA,2013.0,22.2


In [21]:
# Grouping the data by Year and country to get the mean of the values for each year
Profit_Tax_Commercial_Percent_grouped = Profit_Tax_Commercial_Percent.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Profit_Tax_Commercial_Percent_grouped.head(10)

,Year,CountryCode,Value
0,2009.0,ZAF,24.9
1,2010.0,ZAF,24.8
2,2011.0,ZAF,24.0
3,2012.0,ZAF,23.9
4,2013.0,BRA,22.2
5,2013.0,CHN,10.6
6,2013.0,IND,23.7
7,2013.0,RUS,8.5
8,2013.0,ZAF,21.4
9,2014.0,BRA,22.2


In [22]:

# Creating a line plot to visualize the data for the series "Profit tax (% of commercial profits)"
Profit_Tax_Commercial_Percent_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="Profit tax (% of commercial profits)", ylabel="Profit Tax Commercial (%)")


:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)

In [23]:
#From the Private Sector Data, we are interested in the series "Tax payments (number)"

Tax_Payments_Number = private_sector_df.loc[private_sector_df['SeriesName'] == 'Tax payments (number)']
Tax_Payments_Number.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
33684,Tax payments (number),IC.TAX.PAYM,South Africa,ZAF,2009.0,9.00
34529,Tax payments (number),IC.TAX.PAYM,South Africa,ZAF,2010.0,9.00
35374,Tax payments (number),IC.TAX.PAYM,South Africa,ZAF,2011.0,8.00
36219,Tax payments (number),IC.TAX.PAYM,South Africa,ZAF,2012.0,8.00
37060,Tax payments (number),IC.TAX.PAYM,Brazil,BRA,2013.0,9.61


In [24]:
# Grouping the data by Year and country to get the mean of the values for each year
Tax_Payments_Number_grouped = Tax_Payments_Number.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Tax_Payments_Number_grouped.head(10)

,Year,CountryCode,Value
0,2009.0,ZAF,9.00
1,2010.0,ZAF,9.00
2,2011.0,ZAF,8.00
3,2012.0,ZAF,8.00
4,2013.0,BRA,9.61
5,2013.0,CHN,9.00
6,2013.0,IND,41.11
7,2013.0,RUS,9.00
8,2013.0,ZAF,7.00
9,2014.0,BRA,9.61


In [25]:

# Creating a line plot to visualize the data for the series "Tax payments (number)"
Tax_Payments_Number_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="Tax payments (number)", ylabel="Tax Payments")



:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)

In [26]:
#From the Private Sector Data, we are interested in the series "Investment in energy with private participation (current US$)"

Investment_Energy_Private_Participation = private_sector_df.loc[private_sector_df['SeriesName'] == 'Investment in energy with private participation (current US$)']
Investment_Energy_Private_Participation.head()

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
33265,Investment in energy with private participatio...,IE.PPI.ENGY.CD,Brazil,BRA,2009.0,2.362411e+10
33266,Investment in energy with private participatio...,IE.PPI.ENGY.CD,China,CHN,2009.0,2.992000e+09
33267,Investment in energy with private participatio...,IE.PPI.ENGY.CD,India,IND,2009.0,2.450385e+10
33268,Investment in energy with private participatio...,IE.PPI.ENGY.CD,Russian Federation,RUS,2009.0,1.484320e+09
34110,Investment in energy with private participatio...,IE.PPI.ENGY.CD,Brazil,BRA,2010.0,1.183830e+10


In [27]:
# Grouping the data by Year and country to get the mean of the values for each year
Investment_Energy_Private_Participation_grouped = Investment_Energy_Private_Participation.groupby(by=["Year","CountryCode"])["Value"].mean().reset_index()
Investment_Energy_Private_Participation_grouped.head(10)

,Year,CountryCode,Value
0,2009.0,BRA,2.362411e+10
1,2009.0,CHN,2.992000e+09
2,2009.0,IND,2.450385e+10
3,2009.0,RUS,1.484320e+09
4,2010.0,BRA,1.183830e+10
5,2010.0,CHN,4.214700e+08
6,2010.0,IND,3.447451e+10
7,2010.0,RUS,5.981920e+09
8,2010.0,ZAF,6.000000e+06
9,2011.0,BRA,1.691230e+10


In [28]:

# Creating a line plot to visualize the data for the series "Investment in energy with private participation (current US$)"
Investment_Energy_Private_Participation_grouped.pivot(index="Year", columns="CountryCode", values="Value").hvplot(kind="line", title="Investment in energy with private participation (current US$)", ylabel="Investment in Energy with Private")



:NdOverlay   [CountryCode]
   :Curve   [Year]   (value)